In [2]:
!pip install spotipy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import spotipy
print("Spotipy has been successfully installed!")

Spotipy has been successfully installed!


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [10]:
client_id = "12432ee31f704732bf6afde68e88190f"
client_secret = "134d08646a9441d38666fd9df056201a"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [12]:
#searching for a particular artist name

artist_name = ["One Direction"]
result = spot.search(artist_name) 
result['tracks']['items'][1]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4AK6F7OLvEQ5QYCBNiQWHq'},
  'href': 'https://api.spotify.com/v1/artists/4AK6F7OLvEQ5QYCBNiQWHq',
  'id': '4AK6F7OLvEQ5QYCBNiQWHq',
  'name': 'One Direction',
  'type': 'artist',
  'uri': 'spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq'}]

In [13]:
# Extract Spotify albums 
# store Spotify URIs and album names in separate lists for reference

#Extract Artist's uri
OneD_uris = result['tracks']['items'][0]['artists'][0]['uri']
#Pull all of the artist's albums
OneD_albums = spot.artist_albums(OneD_uris, album_type='album')
#Store artist's albums' names' and uris in separate lists
OneD_album_names = []
OneD_album_uris = []
for i in range(len(OneD_albums['items'])):
    OneD_album_names.append(OneD_albums['items'][i]['name'])
    OneD_album_uris.append(OneD_albums['items'][i]['uri'])
    
OneD_album_names
OneD_album_uris

['spotify:album:1gMxiQQSg5zeu4htBosASY',
 'spotify:album:4gCNyS7pidfK3rKWhB3JOY',
 'spotify:album:7p1fX8aUySrBdx4WSYspOu',
 'spotify:album:2sWX3HYnZjPZ9MrH6MFsBt',
 'spotify:album:6cunQQ7YZisYOoiFu2ywIq']

In [ ]:
# Extract all the songs from every album

#Next would be to loop through each album to extract key album track data.

def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(artist_album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

Another way


In [1]:
class GetLyrics_data():
    
     def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id, genius_key):
        self.spotify_client_id = spotify_client_id
        self.spotify_client_secret = spotify_client_secret
        self.user_id = user_id
        self.playlist_id = playlist_id
        self.genius_key = genius_key
        
     def get_playlist_info(self):
        token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
        sp = spotipy.Spotify(token)
        playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
        self.playlist = playlist
        return self.playlist
    
     def get_track_names(self):
        track_names = []
        for song in range(len(self.playlist['items'])):
            track_names.append(self.playlist['items'][song]['track']['name'])
        self.track_names = track_names
        return self.track_names
    
     def get_track_artists(self):
        track_artists = []
        for song in range(len(self.playlist['items'])):
            track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
        self.track_artists = track_artists
        return self.track_artists
        
     def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

     def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
     def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
     def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

     def get_lyrics(self):
        playlist = GetLyrics.get_playlist_info(self)
        track_names = GetLyrics.get_track_names(self)
        track_artists = GetLyrics.get_track_artists(self)
        song_lyrics = []
        for i in range(len(self.track_names)):
            print("\n")
            print(f"Working on track {i}.")
            response = GetLyrics.request_song_info(self, self.track_names[i], self.track_artists[i])
            remote_song_info = GetLyrics.check_hits(self)
            if remote_song_info == None:
                lyrics = None
                print(f"Track {i} is not in the Genius database.")
            else:
                url = GetLyrics.get_url(self)
                lyrics = GetLyrics.scrape_lyrics(self)
                if lyrics == None:
                    print(f"Track {i} is not in the Genius database.")
                else:
                    print(f"Retrieved track {i} lyrics!")
            song_lyrics.append(lyrics)
        return song_lyrics